In [228]:
import pickle

with open('vectorizer.pk', 'rb') as f:
    fitted_dict = pickle.load(f)

In [229]:
import json
from bson.json_util import dumps

with open('news.json', "r", encoding="utf8") as f:
    news = eval(json.loads(dumps(f))[0])

In [230]:
import spacy
import numpy as np

def lemmas(lst):
    return " ".join([token.lemma_ for token in lst])
def entities(lst):
    return " ".join([ent.lemma_ for ent in lst.ents])

nlp = spacy.load("ru_core_news_sm")

title = nlp(news[0]['title'])
body = nlp(news[0]['text'])
doc0 = [news[0]['title'], lemmas(title), entities(title), news[0]['text'], lemmas(body), entities(body)]

X0 = [fitted_dict[i].transform([doc0[i]]).toarray().flatten() for i in range(6)] #6 -> 9

In [232]:
from scipy import spatial

def cscore(cluster_X, doc_X):
    return sum(1 - spatial.distance.cosine(cluster_X[i], doc_X[i]) for i in range(6))

clusters = [(X0, [0])]
for i in range(1, 100):
    title = nlp(news[i]['title'])
    body = nlp(news[i]['text'])
    doc = [news[i]['title'], lemmas(title), entities(title), news[i]['text'], lemmas(body), entities(body)]
    X = [fitted_dict[j].transform([doc[j]]).toarray().flatten() for j in range(6)] #6 -> 9

    max_cscore = np.max([cscore(cluster[0], X) for cluster in clusters])
    max_cscore_arg = np.argmax([cscore(cluster[0], X) for cluster in clusters])

    if max_cscore < 1.2:
        clusters.append((X, [i]))
    else:
        cur_cluster = clusters[max_cscore_arg]
        cluster_size = len(cur_cluster[1])
        for j in range(6):
            clusters[max_cscore_arg][0][j] = (cur_cluster[0][j] * cluster_size + X[j]) / (cluster_size + 1)
        clusters[max_cscore_arg][1].append(i)

C:\Users\Sasha\PycharmProjects\graduate\pyenv\lib\site-packages\scipy\spatial\distance.py:699: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [233]:
for cluster in clusters:
    print(cluster[1])
    for c in cluster[1] :
        print(news[c]['title'])

[0, 25]
«ОВД-Инфо» сообщает о задержании 44 демонстрантов в Екатеринбурге
Противников нового храма в Екатеринбурге ОМОН оттеснил от забора
[1, 5, 7, 10, 12, 19, 21, 22, 26, 32, 35, 44, 78, 99]
Лазарев вышел в финал «Евровидения-2019»
Российский певец Сергей Лазарев вышел в финал Евровидения
Сергей Лазарев вышел в финал «Евровидения»
Лазарев признался в сдаче теста ДНК перед «Евровидением»
Сергей Лазарев вышел в финал «Евровидения»
Яна Рудковская назвала основных соперников Сергея Лазарева на «Евровидении»
Певец Сергей Лазарев вышел в финал Евровидения
Сергей Лазарев прошел в финал «Евровидения»
Лазарев очень волновался: стало известно, кто выступит в финале «Евровидения - 2019»
Лазарев вышел в финал «Евровидения-2019»
Сергей Лазарев вышел в финал «Евровидения»
Представитель России Сергей Лазарев прошел в финал «Евровидения»
Названы соперники Лазарева во втором полуфинале Евровидения-2019
Сергей Лазарев готовится выступить во втором полуфинале «Евровидения»
[2]
17 природных пожаров туша